In [ ]:
import sys
import math
import random
import copy

import pandas as pd
import numpy as np

from tqdm import tqdm
import time

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import animation as animation

import matplotlib as mpl

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import os

!pip install torchmetrics
from torchmetrics.image import StructuralSimilarityIndexMeasure

from PIL import Image, ImageSequence
from skimage.transform import resize

from skimage.metrics import structural_similarity as ssim

import json

directory = ''  # your directory

# from google.colab import drive
# drive.mount('/content/drive')

#### model

In [ ]:
class FireDataset(Dataset):
    def __init__(self, fire_data, seq_len):
        self.fire_data = fire_data
        shape = self.fire_data.shape

        self.number = shape[0]
        self.time_step = shape[1]
        self.frame_len = shape[2]
        self.seq_len = seq_len

    def __len__(self):
        return self.number* (self.time_step - self.seq_len +1)

    def __getitem__(self, idx):
        idx0 = idx//(self.time_step - self.seq_len +1)
        idx1 = idx%(self.time_step - self.seq_len +1)
        input, target = torch.tensor(self.fire_data[idx0][idx1:idx1+3]),  torch.tensor(self.fire_data[idx0][idx1+3:idx1+6])

        return  torch.stack((input, input), dim=1), target


In [ ]:
class ConvLSTMCell(nn.Module):
  def __init__(self, input_dim, hidden_dim, kernel_size):
    super(ConvLSTMCell, self).__init__()

    self.input_dim = input_dim
    self.hidden_dim = hidden_dim
    self.kernel_size = kernel_size

    padding = kernel_size // 2
    self.conv = nn.Conv2d(in_channels=input_dim + hidden_dim,
                out_channels=4 * hidden_dim,
                kernel_size=kernel_size,
                padding=padding,
                bias=True)

  def forward(self, input_tensor, cur_state):
    h_cur, c_cur = cur_state

    combined = torch.cat([input_tensor, h_cur], dim=1)
    combined_conv = self.conv(combined)

    cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)

    i = torch.sigmoid(cc_i)
    f = torch.sigmoid(cc_f)
    o = torch.sigmoid(cc_o)
    g = torch.tanh(cc_g)

    c_next = f * c_cur + i * g
    h_next = o * torch.tanh(c_next)

    return h_next, c_next

  def init_hidden(self, batch_size, image_size):
    height, width = image_size
    return (torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device),
        torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device))


In [ ]:
class EncoderDecoderConvLSTM(nn.Module):
  def __init__(self, input_dim, hidden_dim):
    super(EncoderDecoderConvLSTM, self).__init__()

    self.encoder_1_convlstm = ConvLSTMCell(input_dim=input_dim, hidden_dim=hidden_dim, kernel_size=3)
    self.encoder_2_convlstm = ConvLSTMCell(input_dim=hidden_dim, hidden_dim=hidden_dim, kernel_size=3)
    self.decoder_1_convlstm = ConvLSTMCell(input_dim=hidden_dim, hidden_dim=hidden_dim, kernel_size=3)
    self.decoder_2_convlstm = ConvLSTMCell(input_dim=hidden_dim, hidden_dim=hidden_dim, kernel_size=3)
    self.decoder_CNN = nn.Conv3d(in_channels=hidden_dim, out_channels = 16 , kernel_size=(1, 3, 3), padding=(0, 1, 1))


  def autoencoder(self, x, seq_len, future_step, h_t, c_t, h_t2, c_t2, h_t3, c_t3, h_t4, c_t4):

    outputs = []

    # encoder
    for t in range(seq_len):
      h_t, c_t = self.encoder_1_convlstm(input_tensor=x[:, t, :, :], cur_state=[h_t, c_t])  # we could concat to provide skip conn here
      h_t2, c_t2 = self.encoder_2_convlstm(input_tensor=h_t, cur_state=[h_t2, c_t2])  # we could concat to provide skip conn here

    # encoder_vector
    encoder_vector = h_t2

    # decoder
    for t in range(future_step):
      h_t3, c_t3 = self.decoder_1_convlstm(input_tensor=encoder_vector, cur_state=[h_t3, c_t3])  # we could concat to provide skip conn here
      h_t4, c_t4 = self.decoder_2_convlstm(input_tensor=h_t3, cur_state=[h_t4, c_t4])  # we could concat to provide skip conn here
      encoder_vector = h_t4
      outputs += [h_t4]  # predictions

    outputs = torch.stack(outputs, 1)
    outputs = outputs.permute(0, 2, 1, 3, 4)
    outputs = self.decoder_CNN(outputs)
    #outputs = outputs.permute(0, 2, 1, 3, 4)

    return outputs

  def forward(self, x, future_seq=3, hidden_state=None):

    # find size of different input dimensions
    b, seq_len, _, h, w = x.size()

    # initialize hidden states
    h_t, c_t = self.encoder_1_convlstm.init_hidden(batch_size=b, image_size=(h, w))
    h_t2, c_t2 = self.encoder_2_convlstm.init_hidden(batch_size=b, image_size=(h, w))
    h_t3, c_t3 = self.decoder_1_convlstm.init_hidden(batch_size=b, image_size=(h, w))
    h_t4, c_t4 = self.decoder_2_convlstm.init_hidden(batch_size=b, image_size=(h, w))

    # autoencoder forward
    outputs = self.autoencoder(x, seq_len, future_seq, h_t, c_t, h_t2, c_t2, h_t3, c_t3, h_t4, c_t4)

    return outputs



In [ ]:
def train_convLSTM(train_data, test=False, test_data=None):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    input_dim = 2
    hidden_dim = 128
    kernel_size = 3
    num_layers = 3
    seq_len = 3

    model = EncoderDecoderConvLSTM(input_dim, hidden_dim).to(device)

    criterion = nn.CrossEntropyLoss(weight=None, size_average=None, ignore_index=- 100, reduce=None, reduction='mean', label_smoothing=0.0)
    criterion2 = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=3e-4)

    batch_size = 16
    iter = 5

    fire_data_loader_train = DataLoader(train_data, batch_size=batch_size, shuffle=True)

    if test:
        mse_list = []
        ssim_list = []

    for it in range(iter):
        for batch_no, (batch_input, batch_target) in enumerate(fire_data_loader_train):
            model.train()

            batch_input, batch_target = batch_input.to(device, dtype=torch.float32), batch_target.to(device, dtype=torch.int64)

            optimizer.zero_grad()

            logits = model(batch_input)
            """if batch_no % 100 == 0:
                print(torch.unique(batch_input),torch.unique(batch_target),torch.unique(torch.argmax(logits, dim=1)))"""
            #print(f'logit shape{logits.shape}')
            train_loss = criterion(logits, batch_target)

            train_loss.backward()
            optimizer.step()

            """if batch_no % 50 == 0:
                print('Batch:{:5d}  |  Train loss: {:.4f}'.format(batch_no, train_loss))"""

            if test:
                if batch_no % 200 == 0:

                    n_list=[21*random.randint(0,20)+14 for i in range(16)]

                    input1 = []
                    target = []

                    for n in n_list:
                        input1.append(test_data[n][0].numpy())
                        target.append(test_data[n+6][1].numpy())

                    input1 = torch.tensor(input1)
                    target = torch.tensor(target)
                    input0 = input1.clone()

                    # Validation
                    model.eval()
                    with torch.no_grad():
                        test_input_seq, test_input1_seq, test_target_seq = input0, input1, target
                        np_test_target_seq = test_target_seq.numpy()
                        test_input_seq, test_target_seq, test_input1_seq = test_input_seq.to(device, dtype=torch.float32), test_target_seq.to(device, dtype=torch.float32), test_input1_seq.to(device, dtype=torch.float32)
                        test_predicted_seq = loop_testing(test_input1_seq, 3, model)
                        np_test_predicted_seq = test_predicted_seq.cpu().numpy()
                        test_loss2 = criterion2(test_predicted_seq, test_target_seq)
                        mse_list.append(test_loss2)
                        test_loss3 = ssim(np_test_target_seq, np_test_predicted_seq, data_range=np_test_predicted_seq.max() - np_test_predicted_seq.min(), channel_axis=1)
                        ssim_list.append(test_loss3)

                        print("Iter:{:5d}|Train loss: {:.4f}|Test loss MSE: {:.4f}|Test loss SSIM:{:.4f}".format(batch_no, train_loss,test_loss2, test_loss3))

    """    os.makedirs('/content/trained_models/', exist_ok=True)
    model_path = os.path.join('/content/trained_models/', 'convlstm.pt')
    torch.save(model.state_dict(), model_path)"""
    if test:
        return model, mse_list, ssim_list
    else:
        return model


#### data function

In [ ]:
def count_fire(forest):
    count_list=[]

    i, m, n = np.shape(forest)

    for i1 in range(i):
        count = 0
        for m1 in range(m):
            for n1 in range(n):
                if forest[i1][m1][n1] == 1:
                    count += 1
        count_list.append(count)

    return count_list

In [ ]:
def loop_testing_msessimrpe_1to1(test_data, m):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    input1, target1 = test_data[m]
    _, target3 = test_data[m+3]
    _, target6 = test_data[m+6]
    _, target9 = test_data[m+9]
    _, target12 = test_data[m+12]
    input0 = input1.clone()

    input1 = input1.to(device, dtype=torch.float32).unsqueeze(0)
    target1 = target1.to(device, dtype=torch.float32).unsqueeze(0)
    target3 = target3.to(device, dtype=torch.float32).unsqueeze(0)
    target6 = target6.to(device, dtype=torch.float32).unsqueeze(0)
    target9 = target9.to(device, dtype=torch.float32).unsqueeze(0)
    target12 = target12.to(device, dtype=torch.float32).unsqueeze(0)

    mse_list = []
    ssim_list = []
    rpe_list = []
    pre_fire_count = []
    tar_fire_count = []

    time_list=[]

    mse = nn.MSELoss()

    #loop=1
    old_input = input1
    start = time.time()
    input1 = convLSTM(old_input).detach()
    time_list.append(time.time()-start)
    input1 = torch.argmax(input1, dim=1)

    np_input1 = input1.cpu().squeeze(0).numpy()
    np_target1 = target1.cpu().squeeze(0).numpy()

    for i in range(3):
        test_loss2 = mse(input1[0][i], target1[0][i])
        mse_list.append(test_loss2)

    for i in range(3):
        test_loss3 = ssim(np_target1[i], np_input1[i], data_range=np_input1[i].max() - np_input1[i].min(), channel_axis=0)
        ssim_list.append(test_loss3)

    for i in range(3):
        test_loss4 = np.sum(np_target1[i] != np_input1[i])/(128*128)
        rpe_list.append(test_loss4)

    pre_fire_count.append(count_fire(np_input1))
    tar_fire_count.append(count_fire(np_target1))

    input1 = torch.stack((input1, input1), dim=2)

    #loop=2
    old_input = input1
    start = time.time()
    input1 = convLSTM(old_input).detach()
    time_list.append(time.time()-start)
    input1 = torch.argmax(input1, dim=1)
    np_input1 = input1.cpu().squeeze(0).numpy()
    np_target3 = target3.cpu().squeeze(0).numpy()

    for i in range(3):
        test_loss2 = mse(input1[0][i], target3[0][i])
        mse_list.append(test_loss2)

    for i in range(3):
        test_loss3 = ssim(np_target3[i], np_input1[i], data_range=np_input1[i].max() - np_input1[i].min(), channel_axis=0)
        ssim_list.append(test_loss3)

    for i in range(3):
        test_loss4 = np.sum(np_target3[i] != np_input1[i])/(128*128)
        rpe_list.append(test_loss4)

    pre_fire_count.append(count_fire(np_input1))
    tar_fire_count.append(count_fire(np_target3))

    input1 = torch.stack((input1, input1), dim=2)

    #loop=3
    old_input = input1
    #print(input.shape)
    start = time.time()
    input1 = convLSTM(old_input).detach()
    time_list.append(time.time()-start)
    input1 = torch.argmax(input1, dim=1)
    np_input1 = input1.cpu().squeeze(0).numpy()
    np_target6 = target6.cpu().squeeze(0).numpy()

    for i in range(3):
        test_loss2 = mse(input1[0][i], target6[0][i])
        mse_list.append(test_loss2)

    for i in range(3):
        test_loss3 = ssim(np_target6[i], np_input1[i], data_range=np_input1[i].max() - np_input1[i].min(), channel_axis=0)
        ssim_list.append(test_loss3)

    for i in range(3):
        test_loss4 = np.sum(np_target6[i] != np_input1[i])/(128*128)
        rpe_list.append(test_loss4)

    pre_fire_count.append(count_fire(np_input1))
    tar_fire_count.append(count_fire(np_target6))

    input1 = torch.stack((input1, input1), dim=2)

    #loop=4
    old_input = input1
    start = time.time()
    input1 = convLSTM(old_input).detach()
    time_list.append(time.time()-start)
    input1 = torch.argmax(input1, dim=1)
    np_input1 = input1.cpu().squeeze(0).numpy()
    np_target9 = target9.cpu().squeeze(0).numpy()


    for i in range(3):
        test_loss2 = mse(input1[0][i], target9[0][i])
        mse_list.append(test_loss2)

    for i in range(3):
        test_loss3 = ssim(np_target9[i], np_input1[i], data_range=np_input1[i].max() - np_input1[i].min(), channel_axis=0)
        ssim_list.append(test_loss3)

    for i in range(3):
        test_loss4 = np.sum(np_target9[i] != np_input1[i])/(128*128)
        rpe_list.append(test_loss4)

    pre_fire_count.append(count_fire(np_input1))
    tar_fire_count.append(count_fire(np_target9))

    input1 = torch.stack((input1, input1), dim=2)

    #loop=5
    old_input = input1
    #print(input.shape)
    start = time.time()
    input1 = convLSTM(old_input).detach()
    time_list.append(time.time()-start)
    input1 = torch.argmax(input1, dim=1)
    np_input1 = input1.cpu().squeeze(0).numpy()
    np_target12 = target12.cpu().squeeze(0).numpy()

    for i in range(3):
        test_loss2 = mse(input1[0][i], target12[0][i])
        mse_list.append(test_loss2)

    for i in range(3):
        test_loss3 = ssim(np_target12[i], np_input1[i], data_range=np_input1[i].max() - np_input1[i].min(), channel_axis=0)
        ssim_list.append(test_loss3)

    for i in range(3):
        test_loss4 = np.sum(np_target12[i] != np_input1[i])/(128*128)
        rpe_list.append(test_loss4)

    pre_fire_count.append(count_fire(np_input1))
    tar_fire_count.append(count_fire(np_target12))

    input1 = torch.argmax(input1, dim=1)

    for i in range(len(mse_list)):
        mse_list[i] = float(mse_list[i].cpu())

    return input0, input1, np.array(mse_list), np.array(ssim_list), np.array(rpe_list), pre_fire_count, tar_fire_count, time_list


#### model speed test

change the model path for testing on different models

In [ ]:
# Load the model from the .pt file
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_dim = 2
hidden_dim = 128
convLSTM = EncoderDecoderConvLSTM(input_dim, hidden_dim).to(device)
model_path = os.path.join(directory + 'Data/trained_models/', 'convlstm_ferguson_rand_ig_rand_wind_3t1p.pt') # change the model path for testing on different models
convLSTM.load_state_dict(torch.load(model_path))

# Set the model to evaluation mode (if needed)
convLSTM.eval()

In [ ]:
import json

sizes = [128, 256, 384, 512, 640, 768]
conv_time_dic = {}

for size in sizes:
    print(f'Now we are at size {size}')
    conv_time_dic[f'{size}'] = []
    InfFireDataset = FireDataset(np.random.rand(100,26,size,size), seq_len=6)

    for n in tqdm(range(10)):
        m = 21*n + 6
        _, _, _, _, _, _, _, time_list = loop_testing_msessimrpe_1to1(InfFireDataset, m)
        conv_time_dic[f'{size}'] = conv_time_dic[f'{size}'] + time_list
    # print('\n','size =',size, 'Inference time=', np.mean(mean_time_list))

print(conv_time_dic)

In [ ]:
# Saving the dictionary to a file
# change the file path for testing on different models
with open(directory + 'Data/inference_speed_evaluation_data/fer_t4_conv_time_dic.json', 'w') as json_file:
    json.dump(conv_time_dic, json_file)
